Links to source materials:
* https://medium.com/analytics-vidhya/spam-classification-with-tensorflow-keras-7e9fb8ace263
* https://github.com/Harshita9511/Spam-Classification-with-Tensorflow-Keras

In [18]:
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

In [19]:
df = pd.read_csv('emails.csv')
df = df[['text', 'spam']]
df.head()

,text,spam
0,naturally irresistible your corporate identity...,1
1,the stock trading gunslinger fanny is merrill...,1
2,unbelievable new homes made easy im wanting t...,1
3,4 color printing special request additional i...,1
4,"do not have money , get software cds from here...",1


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


In [21]:
def text_processing(message):
    stop_words = stopwords.words('english')
    no_punctuation = [char for char in message if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stop_words])

df['text'] = df['text'].apply(text_processing)
df.head()

,text,spam
0,naturally irresistible corporate identity lt r...,1
1,stock trading gunslinger fanny merrill muzo co...,1
2,unbelievable new homes made easy im wanting sh...,1
3,4 color printing special request additional in...,1
4,money get software cds software compatibility ...,1


In [22]:
X = df['text'].values
y = df['spam'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [23]:
# Vectorization.
bow = CountVectorizer()
X_train = bow.fit_transform(X_train)
X_test = bow.transform(X_test)

# Term Frequency, Inverse Document Frequency.
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train=X_train.toarray()
X_test=X_test.toarray()

In [24]:
model = Sequential()
model.add(Dense(units=8000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=4000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=400, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model.fit(x=X_train, y=y_train, epochs=5, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stop])

Epoch 1/5
144/144 [==============================] - 108s 745ms/step - loss: 0.2367 - val_loss: 0.1132

In [25]:
predictions = model.predict_classes(X_test)

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

/home/ar/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[854   2]
 [ 12 278]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       856
           1       0.99      0.96      0.98       290

    accuracy                           0.99      1146
   macro avg       0.99      0.98      0.98      1146
weighted avg       0.99      0.99      0.99      1146

